In [74]:
import pandas as pd
import datetime 
old = pd.read_csv('./files/series_Barreto 1_18-21.csv')
jun = pd.read_excel('./files/Barreto 1.xlsx', sheet_name='Junho')
jul = pd.read_excel('./files/Barreto 1.xlsx', sheet_name='Julho')
aug = pd.read_excel('./files/Barreto 1.xlsx', sheet_name='Agosto')
set = pd.read_excel('./files/Barreto 1.xlsx', sheet_name='Setembro')
oct = pd.read_excel('./files/Barreto 1.xlsx', sheet_name='Outubro')

obs=pd.concat([old,oct,jun,jul,aug,set])

/home/gabriel/miniconda3/envs/smap/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [75]:
obs['Hora Leitura'] = pd.to_datetime(obs['Hora Leitura'])
hourly_obs = obs[['Hora Leitura', '01 h']]
hourly_obs


,Hora Leitura,01 h
0,2018-11-30 23:45:00,0.0
1,2018-11-30 23:30:00,0.0
2,2018-11-30 23:15:00,0.0
3,2018-11-30 23:00:00,0.0
4,2018-11-30 22:45:00,0.0
...,...,...
2875,2021-09-30 22:45:00,0.0
2876,2021-09-30 23:00:00,0.0
2877,2021-09-30 23:15:00,0.0
2878,2021-09-30 23:30:00,0.0


In [76]:
hourly_obs_acum01 = hourly_obs.where(obs['Hora Leitura'].dt.minute==0).dropna()


In [77]:
hourly_obs_acum01

,Hora Leitura,01 h
3,2018-11-30 23:00:00,0.0
7,2018-11-30 22:00:00,0.0
11,2018-11-30 21:00:00,0.0
15,2018-11-30 20:00:00,0.0
19,2018-11-30 19:00:00,0.0
...,...,...
2860,2021-09-30 19:00:00,0.0
2864,2021-09-30 20:00:00,0.0
2868,2021-09-30 21:00:00,0.0
2872,2021-09-30 22:00:00,0.0


# Para UTC: BR time zone +3h 

In [78]:
import pandas as pd
import datetime
obs = hourly_obs_acum01
obs

,Hora Leitura,01 h
3,2018-11-30 23:00:00,0.0
7,2018-11-30 22:00:00,0.0
11,2018-11-30 21:00:00,0.0
15,2018-11-30 20:00:00,0.0
19,2018-11-30 19:00:00,0.0
...,...,...
2860,2021-09-30 19:00:00,0.0
2864,2021-09-30 20:00:00,0.0
2868,2021-09-30 21:00:00,0.0
2872,2021-09-30 22:00:00,0.0


In [79]:

obs['Hora Leitura'] = obs['Hora Leitura'].apply(lambda x: x + datetime.timedelta(hours=3))
obs.to_csv("./files/UTC_series_Barreto_18-21.csv")

In [80]:
obs['Hora Leitura']

3      2018-12-01 02:00:00
7      2018-12-01 01:00:00
11     2018-12-01 00:00:00
15     2018-11-30 23:00:00
19     2018-11-30 22:00:00
               ...        
2860   2021-09-30 22:00:00
2864   2021-09-30 23:00:00
2868   2021-10-01 00:00:00
2872   2021-10-01 01:00:00
2876   2021-10-01 02:00:00
Name: Hora Leitura, Length: 25436, dtype: datetime64[ns]

In [38]:
obs.where(obs['Hora Leitura']==datetime.date(2021,10,1)).dropna()

,Hora Leitura,01 h


In [9]:
import pandas as pd
import numpy as np
import datetime

In [10]:
def rain_values_treat(val):
    """small functionality to convert str rain values to float
        used with INMET observed data
    """
    if str(val) == 'nan':
        return np.nan 
    else:
        val = val.replace(',','.')
        return float(val)


def converting_int_to_hours(val):
    """small functionality to convert int hour values to string
        used with INMET observed data
    """
    str_val = str(val).split('.')[0]
    if len(str_val) == 1:
        str_val = '0' + str_val

    hour_val = str_val + ':00:00' 
    return hour_val

In [11]:
obs = pd.read_csv('/home/github/RNA_Operacional_WRF/files/obs_data/INMETobserved_01-2022_05-2022.csv', sep=';')
obs_rain = obs[["Data","Hora (UTC)","Chuva (mm)"]]

obs_rain['Hora (UTC)'] = obs_rain['Hora (UTC)']/100
obs_rain['Hora (UTC)'] = obs_rain['Hora (UTC)'].apply(lambda x: converting_int_to_hours(x))
obs_rain['data'] = obs_rain['Data'] + ' ' + obs_rain['Hora (UTC)']
obs_rain['data'] = pd.to_datetime(obs_rain['data'], format='%Y-%m-%d %H:%M:%S')

obs_rain['Chuva (mm)'] = obs_rain['Chuva (mm)'].apply(lambda x: rain_values_treat(x))
obs_rain['Chuva (mm)'] = obs_rain['Chuva (mm)'].interpolate()

obs_rain.drop(['Data','Hora (UTC)'], axis=1, inplace=True)
obs_rain.rename(columns={'Chuva (mm)': 'chuva 1h'}, inplace=True)


<ipython-input-11-f721ab987e92>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_rain['Hora (UTC)'] = obs_rain['Hora (UTC)']/100
<ipython-input-11-f721ab987e92>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_rain['Hora (UTC)'] = obs_rain['Hora (UTC)'].apply(lambda x: converting_int_to_hours(x))
<ipython-input-11-f721ab987e92>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [12]:
obs_rain

,chuva 1h,data
0,0.0,2022-01-01 00:00:00
1,0.2,2022-01-01 01:00:00
2,0.4,2022-01-01 02:00:00
3,0.2,2022-01-01 03:00:00
4,0.0,2022-01-01 04:00:00
...,...,...
2899,0.0,2022-05-01 19:00:00
2900,0.0,2022-05-01 20:00:00
2901,0.0,2022-05-01 21:00:00
2902,0.0,2022-05-01 22:00:00
